In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def get_products_in_page(page):
    list_prod = page.findAll(attrs= {"vocab":"http://schema.org/","class": "row", "typeof": "Product"})
    list_prod = list_prod[0].findAll('div', attrs={'class':'row'})
    prods=[]
    for i in list_prod:
        url_image = i.findAll('img')[0]['src'].replace('ch/C','gr/G')
        title = i.findAll('img')[0]['alt']
        price = float(i.findAll('h2',attrs={"property": "price"})[0].text.strip("$ "))
        prods.append({"title":title,"url_image": url_image,"proce":price})
    return prods

def get_products_with_delay(brower, max_delay):
    for i in range(max_delay):
        page = BeautifulSoup(browser.page_source,"html5lib")
        products = get_products_in_page(page)
        if products:
            break
        time.sleep(1)
    return products

In [2]:
catalog_url = 'https://www.farmaciasdesimilares.com/#!/medicamentos'
browser = webdriver.Chrome('./chromedriver')
browser.get(catalog_url)
browser.implicitly_wait(10)

In [9]:
for i in range(1,82):
    products = get_products_with_delay(brower, 5)
    print("scrappend page: {}".format(i))

1
2
3
4
5
6
7


In [4]:
#browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[3]/a').click()
browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[9]/a').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[9]/a').click()

In [5]:
page = BeautifulSoup(browser.page_source,"html5lib")
products = get_products_in_page(page)

In [10]:
products

[{'title': 'LORATADINA / BETAMETASONA TABLETAS',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_1187_1.jpg',
  'proce': 42.0},
 {'title': 'IBUPROFENO TABLETAS',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_119_1.jpg',
  'proce': 22.0},
 {'title': 'LEVODROPROPIZINA SOLUCION',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_1194.jpg',
  'proce': 63.0},
 {'title': 'LATANOPROST SOLUCION OFTALMICA',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_1195.jpg',
  'proce': 99.0},
 {'title': 'RANITIDINA JARABE',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_1196.jpg',
  'proce': 80.0},
 {'title': 'TIBOLONA TABLETAS',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_1197.jpg',
  'proce': 149.0},
 {'title': 'ALBENDAZOL TABLETAS',
  'url_image': 'https://farmaciasdesimilares.com/ui/productos/gr/G_12.jpg',
  'proce': 10.0},
 {'title': 'ISOSORBIDA TABLETAS',
  'url_image': 'https://fa

In [12]:
def get_actions_id_auto_increment(data_list, index, doctype):
    return [{
        "_index": index,
        "_id": _id,
        "_type": doctype,
        "_source": i
    } for i in data_list]

In [13]:
get_actions_id_auto_increment(products, 'drugstore', 'document')

NameError: name '_id' is not defined

In [ ]:
def generate_ids(df):
    def increase():
        increase.i += 1
        return increase.i

    increase.i = DataQuality().get_last_id()
    df['id'] = df['id'].apply(lambda _: increase())
    return df

NameError: name 'es' is not defined